# 🧠 Proyecto de Detección de Fraude con DNN
Este notebook está diseñado para ejecutarse directamente en Google Colab y permite clonar el repositorio, instalar dependencias, procesar datos, entrenar una red neuronal y evaluar su desempeño.

In [ ]:
# 📥 Clonar el repositorio (si es público)
!git clone https://github.com/solivare/DeepNeuralNetworkUSS.git
%cd DeepNeuralNetworkUSS/Examples/Fraud

In [ ]:
# 📦 Instalar dependencias necesarias
!pip install -r requirements.txt

In [ ]:
# 🧪 Procesar datos según configuración
!python src/preprocess.py

In [ ]:
# 🚀 Entrenar el modelo definido en config.yaml
!python src/train.py

In [ ]:
# 📊 Evaluación del modelo entrenado
import sys, os
sys.path.append('src')
import train, evaluate
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from utils import plot_score_distribution, plot_training, get_training_data_path

# Cargar configuración y datos
config = train.load_config()
train_path = get_training_data_path(config)
X_train, y_train = train.load_data(train_path, config['base_dir'])
X_test, y_test = train.load_data(config['paths']['test'], config['base_dir'])

# Escalar
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Cargar modelo y predecir
model = load_model('models/model.keras')
y_prob = model.predict(X_test).flatten()
y_pred = (y_prob > 0.5).astype(int)

# Métricas
evaluate.print_classification_metrics(y_test, y_pred, y_prob)
plot_score_distribution(y_test, y_prob)